# Politecnico Malvinas Argentinas- PDH: Preprocesamiento, Normalización y Técnicas Básicas

## Análisis del Código Original

### Propósito de cada etapa del código:

1. **Instalación de librerías y carga de modelos**  
   Se instalan y cargan las librerías necesarias (`nltk`, `spacy`, `sklearn`, etc.) junto con el modelo en español de spaCy (`es_core_news_md`), fundamental para tareas de análisis lingüístico como lematización, POS tagging o reconocimiento de entidades.

2. **Descarga de recursos de NLTK**  
   Se descargan recursos como `punkt` y `stopwords`, necesarios para la tokenización y eliminación de palabras vacías.

3. **Preprocesamiento del texto**  
   Se aplican pasos clásicos de limpieza: convertir a minúsculas, eliminar signos de puntuación, eliminar acentos (`unidecode`), tokenizar, remover stopwords y realizar stemming/lematización. También se aplican expresiones regulares para limpiar caracteres especiales.

4. **Medición de similitud entre textos**  
   Se utilizan métricas como `fuzzywuzzy`, `edit_distance` y `Jaro-Winkler` para calcular similitudes entre pares de palabras o frases.

5. **Representación vectorial del texto**  
   Se usan técnicas como Bag of Words, TF-IDF y N-gramas para convertir los textos en vectores numéricos, facilitando el análisis cuantitativo posterior.

6. **Análisis exploratorio con spaCy**  
   Se aplican técnicas más avanzadas como POS tagging, Named Entity Recognition (NER) y dependencias sintácticas, todo soportado por el modelo de spaCy en español.

---

### Limitaciones y posibles mejoras:

- **Redundancia entre NLTK y spaCy**: Puede simplificarse unificando las tareas de NLP con una sola herramienta (preferentemente spaCy por su robustez).
- **Dataset demasiado pequeño**: Limita el análisis estadístico y la generalización de los resultados.
- **Falta de métricas cuantitativas**: Sería útil agregar métricas para evaluar el impacto de los pasos de preprocesamiento, como el tamaño del vocabulario antes/después o similitud media entre frases.
- **Falta de separación entre entrenamiento y análisis**: Para un enfoque más realista, conviene separar datos de entrada, validación y análisis.
- **Ausencia de modelos semánticos avanzados**: No se aplican técnicas modernas como embeddings (Word2Vec, GloVe) o modelos como BERT que podrían capturar relaciones más profundas entre palabras.


In [1]:
# Instalación de librerías necesarias (si es necesario en Colab)
!pip install nltk spacy scikit-learn thefuzz stop_words
!pip install fuzzywuzzy python-Levenshtein

import re
import nltk
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Cargar modelo en español de spaCy
!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
      --------------------------------------- 0.8/42.3 MB 2.4 MB/s eta 0:00:18
     - -------------------------------------- 1.6/42.3 MB 3.4 MB/s eta 0:00:13
     -- ------------------------------------- 2.4/42.3 MB 3.8 MB/s eta 0:00:11
     -- ------------------------------------- 3.1/42.3 MB 3.4 MB/s eta 0:00:12
     --- ------------------------------------ 3.7/42.3 MB 3.4 MB/s eta 0:00:12
     --- ------------------------------------ 4.2/42.3 MB 3.2 MB/s eta 0:00:12
     ---- ----------------------------------- 5.0/42.3 MB 3.2 MB/s eta 0:00:12
     ----- ---------------------------------- 5.8/42.3 MB 3.4 MB/s eta 0:00:11
     ------ --------------------------------- 6.6/42.3 MB 3.3 MB/s eta 0:00:11
     ------ --------------------------------- 6.8/42.3 MB 3.2 MB/s eta 0:00:11
     ------- -------------------------------- 7.6/42.3 MB 3.1 MB/s

# 1. Preprocesamiento de texto

# Tokenizacion

Quizás una de las operaciones más básicas que podemos hacer es contar palabras. Imaginemos un caso extremadamente sencillo relacionado al aprendizaje automático. Pensemos en un clasificador de positividad / negatividad.

Podríamos usar una regla como:

si "bueno" está en el texto, clasificamos como positivo.

Claramente un enfoque tan sencillo va a ser propenso a muchos errores. Iremos viendo cómo aplicarlo y mejorarlo.

La operación de separar los textos en unidades básicas (o tokens) la llamamos tokenización.

In [4]:
doc2 = 'Usualmente, existe una relación costo-beneficio entre las distintas técnicas.'
doc2.split(' ')

['Usualmente,',
 'existe',
 'una',
 'relación',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'técnicas.']

Vemos que en castellano, el lenguaje natural (escrito) puede tener distintas fuentes de variaciones:

Mayúsculas
Tildes
Signos ortográficos
Errores de tipeo
Variaciones propias del lenguaje como la conjugación
Artefactos de la escritura informal como "holaaa"
¿Cómo abordarán esto nuestros algorítmos?

Una solución es la normalización: con acepción similar aunque distinta a la que usamos en estadística, va a significar transformar a los strings a representaciones iguales.

Podemos:

Sacar espacios
Pasar a minúsculas
Quitar tildes

In [5]:
%%capture
!pip install unidecode

In [6]:
import unidecode
[unidecode.unidecode(w.lower().strip()) for w in doc2.split(' ')]

['usualmente,',
 'existe',
 'una',
 'relacion',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'tecnicas.']

Otras normalizaciones más sofisticadas serían:
- Pasar de plural a singular
- Convertir el género de la palabra
- Conjugar los verbos en infinitivo

¡Es de esperarse que esto requiera métodos más complejos!

Vemos que con la tokenización anterior, todavía tenemos palabras con signos de puntuación pegados. Armar nuestro vocabulario es un paso clave para nuestros modelos de *Machine Learning*. 

In [7]:

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = word_tokenize(text.lower())
print("Tokenización y Normalización:", words)




Tokenización y Normalización: ['¡hola', '!', '¿cómo', 'estás', '?', 'estoy', 'aprendiendo', 'nlp', 'con', 'spacy', 'y', 'nltk', '.', 'nlp', 'es', 'fascinante', '.']


In [8]:
# Expresiones Regulares
text_clean = re.sub(r'[^a-zA-Záéíóúüñ ]', '', text.lower())
print("Texto limpio con regex:", text_clean)



Texto limpio con regex: hola cómo estás estoy aprendiendo nlp con spacy y nltk nlp es fascinante


In [9]:
# Stop Words
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

Sin Stop Words: ['¡hola', '!', '¿cómo', '?', 'aprendiendo', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


# 2. Técnicas de NLP con NLTK y spaCy

In [10]:
# Stemming con NLTK
stemmer = SnowballStemmer("spanish")
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print("Stemming:", stemmed_words)


Stemming: ['¡hol', '!', '¿com', '?', 'aprend', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascin', '.']


In [11]:
# Lemmatización con spaCy
doc = nlp(" ".join(filtered_words))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['¡', 'hola', '!', '¿', 'cómo', '?', 'aprender', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


In [12]:
# Distancias entre strings
print("Distancia fuzzy entre 'casa' y 'casas':", fuzz.ratio("casa", "casas"))
texts = ["El gato come pescado", "El perro ladra en la casa"]

Distancia fuzzy entre 'casa' y 'casas': 89


In [13]:
# segundo ejemplo %%capture
!pip install pyjarowinkler
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist

In [14]:
# Definimos una lista de tuplas de palabras
palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"),  ("totalmente","diferentes"), ("pero", "basta")]

# Calculamos las metricas de distancia pasando cada tupla como argumentos a levdist() y get_jaro_distance()
for x,y in palabras:
    print(f"'{x}' vs '{y}':")
    print("Distancia Levenshtein ->", edit_distance(x,y))
    print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y))
    print("-"*40)

'pero' vs 'perro':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.07
----------------------------------------
'pero' vs 'pierdo':
Distancia Levenshtein -> 2
Similitud Jaro Winkler -> 0.11
----------------------------------------
'nueve' vs 'mueve':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.13
----------------------------------------
'totalmente' vs 'diferentes':
Distancia Levenshtein -> 7
Similitud Jaro Winkler -> 0.48
----------------------------------------
'pero' vs 'basta':
Distancia Levenshtein -> 5
Similitud Jaro Winkler -> 1.0
----------------------------------------


# 3. Vectorización y Representación del Texto

In [15]:
texts = ["El gato come pescado", "El perro ladra en la casa"]

In [16]:
# Bolsa de Palabras (BoW)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


casa  come  el  en  gato  la  ladra  perro  pescado
0     0     1   1   0     1   0      0      0        1
1     1     0   1   1     0   1      1      1        0

In [17]:
# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(texts)
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

casa      come        el       en      gato       la    ladra    perro  \
0  0.00000  0.534046  0.379978  0.00000  0.534046  0.00000  0.00000  0.00000   
1  0.42616  0.000000  0.303216  0.42616  0.000000  0.42616  0.42616  0.42616   

    pescado  
0  0.534046  
1  0.000000

In [18]:
# N-gramas
vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1)
X_ngram = vectorizer_ngram.fit_transform(texts)
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())


come pescado  el gato  el gato come  el perro  el perro ladra  en la  \
0             1        1             1         0               0      0   
1             0        0             0         1               1      1   

   en la casa  gato come  gato come pescado  la casa  ladra en  ladra en la  \
0           0          1                  1        0         0            0   
1           1          0                  0        1         1            1   

   perro ladra  perro ladra en  
0            0               0  
1            1               1

# 4. Tareas Avanzadas con spaCy

In [19]:
sentence = "El presidente de Argentina visitó Buenos Aires."
doc = nlp(sentence)


In [20]:
# POS Tagging
print([(token.text, token.pos_) for token in doc])

[('El', 'DET'), ('presidente', 'NOUN'), ('de', 'ADP'), ('Argentina', 'PROPN'), ('visitó', 'VERB'), ('Buenos', 'PROPN'), ('Aires', 'PROPN'), ('.', 'PUNCT')]


In [21]:
# Named Entity Recognition (NER)
print([(ent.text, ent.label_) for ent in doc.ents])

[('Argentina', 'LOC'), ('Buenos Aires', 'LOC')]


In [22]:
# Dependencias sintácticas
displacy.render(doc, style='dep', jupyter=True)

NameError: name 'displacy' is not defined

In [23]:

# Limitaciones de Bolsa de Palabras
print("BoW no captura el significado de las palabras ni el orden de las mismas.")

BoW no captura el significado de las palabras ni el orden de las mismas.
